## Import Libraries

In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [4]:
from os import getcwd, path, pardir

## Compile and program project

In [5]:
project_path = path.join(getcwd(), "BitCounting-S")
project_path

'C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Experiments\\Workloads\\BitCounting\\BitCounting-S'

In [6]:
atprogram(project_path, verbose=2)

make: Nothing to be done for 'all'.

[DEBUG] Starting execution of "chiperase"
[DEBUG] Starting process 'C:\Program Files (x86)\Atmel\Studio\7.0\atbackend\atbackend.exe'
[DEBUG] Connecting to TCP:127.0.0.1:4011
[INFO] Connected to edbg, fw version: 3.25
[INFO] Firmware check OK
[DEBUG] Command "chiperase" finished with return code 0
[DEBUG] Starting execution of "program"
[DEBUG] Memory segment base written at 0x00000000. Size = 0x000008c0.
[DEBUG] Memory segment base written at 0x000008c0. Size = 0x00000064.
[DEBUG] Memory segment base written at 0x0000fb00. Size = 0x00000020.
[DEBUG] Memory segment base written at 0x00804000. Size = 0x00000020.
[DEBUG] Command "program" finished with return code 0
[DEBUG] Exit successfully.
Firmware check OK
Chiperase completed successfully
Programming completed successfully.



0

## Data Logging

In [7]:
live_plot = True

Create a figure for the plot.

In [8]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [9]:
if live_plot:
    config_dict = {
        "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
        "plot_pins": [False, False, True, True],
        "gpio_delay_time" : 0.0007,
        "plot_pins_method": "line",
        "plot_xmax": 1,
        "fig": fig,
        "window_title": "Experiment BitCounting",
        "file_name_base": "experiment_bit_counting"
    }
else:
    config_dict = {
        "loggers": [LOGGER_OBJECT, LOGGER_CSV],
        "gpio_delay_time" : 0.0007,
        "file_name_base": "experiment_bit_counting"
    }

Stop criteria to pass to the logger:

In [10]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Stop criteria to pass to the parser:

In [11]:
def stop_function(pin_values):
    return all(pin_values)

In [12]:
repetitions = 10

Perform the measurement.

In [13]:
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
        
charges = []
times = []

with DGILibExtra(**config_dict) as dgilib:
    for _ in range(repetitions):
        dgilib.device_reset()
        dgilib.logger.log(2, stop_fn)
        print(dgilib.data)
        charge, time = power_and_time_per_pulse(dgilib.data, 2, 0, stop_function=stop_function)
        charges += charge
        times += time
        if len(charge) > 1 or len(time) > 1:
            print(f"Parsing of measurement data failed, charge:{charge}, time:{time}")
            break
        dgilib.logger.plotobj.ax.set_title(f"Previous sample: charge: {charge[0]*1e3:.06} mC, time: {time[0]:.06} s")
        dgilib.empty_data()
    
    dgilib.logger.plotobj.ax.set_title(f"Average of {repetitions} samples: charge: {sum(charges)/len(charges)*1e3:.06} mC, time: {sum(times)/len(times):.06} s")

D:\Anaconda3\envs\SAML11-32\lib\site-packages\matplotlib\widgets.py:346: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=0, right=0
  ax.set_xlim((valmin, valmax))


Interfaces:
	  48:   gpio,    samples:       8
	 256:   power,   samples:   29000

532
[6.624934582096291e-06] [0.022706018133333325]
Interfaces:
	  48:   gpio,    samples:       8
	 256:   power,   samples:   31000

531
[6.610974056289986e-06] [0.02263946346666667]
Interfaces:
	  48:   gpio,    samples:       8
	 256:   power,   samples:   28000

513
[6.618493822493685e-06] [0.022640195199999998]
Interfaces:
	  48:   gpio,    samples:       8
	 256:   power,   samples:   30000

534
[6.630402723470277e-06] [0.022705284266666664]
Interfaces:
	  48:   gpio,    samples:       8
	 256:   power,   samples:   31000

529
[6.5921970595579785e-06] [0.0226390976]
Interfaces:
	  48:   gpio,    samples:       8
	 256:   power,   samples:   31000

529
[6.623043446220665e-06] [0.0226390976]
Interfaces:
	  48:   gpio,    samples:       8
	 256:   power,   samples:   31000

527
[6.616656388818032e-06] [0.022639463466666666]
Interfaces:
	  48:   gpio,    samples:       8
	 256:   power,   samples:   29

In [14]:
import pandas as pd

In [15]:
df = pd.DataFrame({'times': times, 'charges':charges})

In [16]:
df

,times,charges
0,0.022706,0.000007
1,0.022639,0.000007
2,0.022640,0.000007
3,0.022705,0.000007
4,0.022639,0.000007
5,0.022639,0.000007
6,0.022639,0.000007
7,0.022637,0.000007
8,0.022703,0.000007
9,0.022703,0.000007


In [17]:
df.describe()

,times,charges
count,10.000000,1.000000e+01
mean,0.022665,6.619365e-06
std,0.000034,1.219877e-08
min,0.022637,6.592197e-06
25%,0.022639,6.614269e-06
50%,0.022640,6.620769e-06
75%,0.022703,6.627945e-06
max,0.022706,6.634523e-06


In [18]:
sum(charges)/repetitions

6.619364574372876e-06

In [19]:
print("%.0100f" % (sum(charges)/repetitions))

0.0000066193645743728758915470737622666064225995796732604503631591796875000000000000000000000000000000


In [20]:
NUM = 2048
ones = "".join([bin(n) for n in range(NUM)]).count('1')

In [21]:
-2 + 2 > 1

False